<a href="https://colab.research.google.com/github/Suryanshagrawal19/Proof_Generation/blob/main/transformer%20(greedy_approach).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torch.nn as nn
from torch.nn import Transformer
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedModel, PretrainedConfig, AutoTokenizer
import pandas as pd  # To load the Parquet dataset
from tqdm import tqdm  # For progress bar
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score

# Helper function: Compute token-level exact match accuracy.
def get_match_accuracy(generated: str, actual: str) -> float:
    """
    Computes the percentage of tokens in the actual proof that match the tokens in the generated proof (in order).
    Returns a percentage value.
    """
    gen_tokens = generated.split()
    actual_tokens = actual.split()
    if not actual_tokens:
        return 0.0
    # Count tokens that match in the same position.
    matches = sum(1 for gt, at in zip(gen_tokens, actual_tokens) if gt == at)
    return (matches / len(actual_tokens)) * 100

# 1. Load Parquet Dataset Function
def load_dataset(file_path="combined_dataset.parquet"):
    try:
        df = pd.read_parquet(file_path)
        proof_pairs = list(zip(df["Output"], df["Theorem"]))  # Adjust column names as needed.
        print(f"Loaded {len(proof_pairs)} examples from {file_path}.")
        return proof_pairs
    except Exception as e:
        print("Error loading dataset:", e)
        # Return a small dummy dataset if the file is not found.
        return [("axiom A -> B", "theorem A implies B"), ("forall x, P(x)", "universal quantifier P(x)")]

# 2. Model & Configuration Classes
class ProofTransformerConfig(PretrainedConfig):
    def __init__(self, vocab_size=30522, d_model=512, num_layers=8, nhead=8,
                 dim_feedforward=4096, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.num_layers = num_layers
        self.nhead = nhead
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout

class ProofTransformer(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.token_embedding = nn.Embedding(config.vocab_size, config.d_model)
        # Using a fixed maximum sequence length of 1000 for positional encoding.
        self.positional_encoding = nn.Parameter(torch.zeros(1, 1000, config.d_model))

        self.encoder = Transformer(
            d_model=config.d_model,
            nhead=config.nhead,
            num_encoder_layers=config.num_layers,
            num_decoder_layers=config.num_layers,
            dim_feedforward=config.dim_feedforward,
            dropout=config.dropout,
        )

        self.lm_head = nn.Linear(config.d_model, config.vocab_size)
        self.init_weights()

    def forward(self, input_ids, decoder_input_ids):
        src = self.token_embedding(input_ids) + self.positional_encoding[:, :input_ids.shape[1], :]
        tgt = self.token_embedding(decoder_input_ids) + self.positional_encoding[:, :decoder_input_ids.shape[1], :]

        # Transformer expects input as [sequence_length, batch_size, d_model]
        src = src.transpose(0, 1)
        tgt = tgt.transpose(0, 1)

        output = self.encoder(src, tgt)

        output = output.transpose(0, 1)
        logits = self.lm_head(output)
        return {"logits": logits}

# 3. Dataset, DataLoader, Tokenizer
class ProofDataset(Dataset):
    def __init__(self, tokenizer, proof_pairs, max_length=512):
        self.tokenizer = tokenizer
        self.data = proof_pairs
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text, target_text = self.data[idx]
        input_ids = self.tokenizer.encode(
            input_text, add_special_tokens=True,
            return_tensors="pt", max_length=self.max_length, truncation=True
        ).squeeze(0)
        target_ids = self.tokenizer.encode(
            target_text, add_special_tokens=True,
            return_tensors="pt", max_length=self.max_length, truncation=True
        ).squeeze(0)
        # Return as tuple of dictionaries (each representing one side of the pair)
        return {"input_ids": input_ids}, {"input_ids": target_ids}

def collate_fn(batch):
    # Since __getitem__ returns a tuple (input_dict, target_dict), we unpack here:
    inputs, targets = zip(*batch)
    input_ids = [item["input_ids"] for item in inputs]
    target_ids = [item["input_ids"] for item in targets]
    input_ids = nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=0)
    target_ids = nn.utils.rnn.pad_sequence(target_ids, batch_first=True, padding_value=0)
    return {"input_ids": input_ids, "target_ids": target_ids}

# 4. Greedy Decoding Function
def greedy_decode(logits):
    # Select the token with the highest probability at each time step.
    token_ids = torch.argmax(logits, dim=-1)
    # Return the tokens for the first sequence in the batch.
    return token_ids[0].cpu().tolist()

# 5. Evaluation Function with Additional Metrics (ROUGE, METEOR, and Match Accuracy)
def evaluate_accuracy(model, dataloader, tokenizer, device):
    model.eval()
    correct = 0
    total = 0
    bleu_scores = []
    meteor_scores = []
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scorer_values = []
    match_accuracies = []  # List to store match accuracies for each proof

    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(dataloader), desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            target_ids = batch["target_ids"].to(device)

            outputs = model(input_ids=input_ids, decoder_input_ids=target_ids)
            logits = outputs["logits"]

            # Use greedy decoding for evaluation on the first example of the batch.
            decoded_tokens = greedy_decode(logits)
            generated_proof = tokenizer.decode(decoded_tokens, skip_special_tokens=True)
            actual_proof = tokenizer.decode(target_ids[0], skip_special_tokens=True)

            # Print proofs and calculate match accuracy.
            print(f"Generated Proof: {generated_proof}")
            print(f"Actual Proof:    {actual_proof}")
            match_acc = get_match_accuracy(generated_proof, actual_proof)
            match_accuracies.append(match_acc)
            print(f"Match Accuracy:  {match_acc:.2f}%")
            print("-" * 80)

            # Exact match evaluation (full string exact match).
            if generated_proof == actual_proof:
                correct += 1

            # Compute BLEU score for semantic similarity.
            bleu_score = sentence_bleu([actual_proof.split()], generated_proof.split())
            bleu_scores.append(bleu_score)

            # Compute METEOR score (using tokenized strings).
            meteor_scores.append(meteor_score([actual_proof.split()], generated_proof.split()))

            # Compute ROUGE score (sum of f-measures from rouge1 and rougeL).
            rouge_score_dict = rouge_scorer_obj.score(actual_proof, generated_proof)
            rouge_total = rouge_score_dict['rouge1'].fmeasure + rouge_score_dict['rougeL'].fmeasure
            rouge_scorer_values.append(rouge_total)

            total += 1

    if total == 0:
        print("No examples evaluated in the test set. Check your test data slice.")
        return

    accuracy = (correct / total * 100) if total > 0 else 0
    avg_bleu = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    avg_meteor = sum(meteor_scores) / len(meteor_scores) if meteor_scores else 0
    avg_rouge = sum(rouge_scorer_values) / len(rouge_scorer_values) if rouge_scorer_values else 0
    avg_match_acc = sum(match_accuracies) / len(match_accuracies) if match_accuracies else 0

    print(f"\nExact Match Accuracy: {accuracy:.2f}%")
    print(f"Average BLEU Score:   {avg_bleu:.4f}")
    print(f"Average METEOR Score: {avg_meteor:.4f}")
    print(f"Average ROUGE Score:  {avg_rouge:.4f}")
    print(f"Average Match Accuracy: {avg_match_acc:.2f}%")

# 6. Initialize Tokenizer, Model, Optimizer, and set Device
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

config = ProofTransformerConfig()
model = ProofTransformer(config)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
epochs = 28
reward_weight = 0.1

# Load dataset from Parquet file
proof_data = load_dataset("combined_dataset.parquet")

# 7. Split dataset into training and testing examples.
# Correcting the slice for test data: using indices 2000 to 2400 for a 400-example test set.
train_data = proof_data[:2000]
test_data = proof_data[2000:2400]

train_dataset = ProofDataset(tokenizer, train_data, max_length=512)
test_dataset = ProofDataset(tokenizer, test_data, max_length=512)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

# 8. Training Loop with Gradient Accumulation
accumulation_steps = 4  # Accumulate gradients over 4 steps (simulates a larger effective batch size)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch+1}/{epochs}", total=len(train_dataloader))

    # Zero gradients outside the loop to begin accumulation.
    optimizer.zero_grad()

    for batch_idx, batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        target_ids = batch["target_ids"].to(device)

        # Forward pass.
        outputs = model(input_ids=input_ids, decoder_input_ids=target_ids)
        logits = outputs["logits"]

        # Compute cross-entropy loss. Flatten the tensors so that we can compare predictions with targets.
        ce_loss = loss_fn(logits.view(-1, config.vocab_size), target_ids.view(-1))
        ce_loss.backward()

        # Gradient accumulation step.
        if (batch_idx + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        running_loss += ce_loss.item()
        progress_bar.set_postfix({"loss": ce_loss.item()})

    # If there are leftover gradients, update once more.
    if (batch_idx + 1) % accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{epochs} completed. Avg Loss: {avg_loss:.4f}")

# 9. Evaluate the model on the test set
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
evaluate_accuracy(model, test_dataloader, tokenizer, device)


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Loaded 100000 examples from combined_dataset.parquet.


Epoch 1/28: 100%|██████████| 250/250 [01:59<00:00,  2.10it/s, loss=3.4]


Epoch 1/28 completed. Avg Loss: 4.9796


...

Generated Proof: lemma avg _set _ no _mma var : ( bp sn ) < = _. a - > av < > 0.
Actual Proof:    lemma avg _ naive _ no _ underflow : ( bpow emin ) < = rabs a - > av < > 0.
Match Accuracy:  8.70%
--------------------------------------------------------------------------------
Generated Proof: lemma avg _ half _ sub _ between : lein x y < = av < = le _ x y.
Actual Proof:    lemma avg _ half _ sub _ between : rmin x y < = av < = rmax x y.
Match Accuracy:  80.00%
--------------------------------------------------------------------------------


Evaluating: 49it [00:10,  6.14it/s]

Generated Proof: lemma avg _ half _ sub _ correct _ aux2 : forall u v,1 u - >1 v - > u < = v - > ( 0 < = u / \ 0 < = v ) \ / ( u < = 0 / \ v < = 0 ) - > _ ( avg _ half _ sub u v - ( ( u + v ) / 2 ) ) < = 3 / 2 * ]p _ flt ( ( u + v ) / 2 ).
Actual Proof:    lemma avg _ half _ sub _ correct _ aux2 : forall u v, format u - > format v - > u < = v - > ( 0 < = u / \ 0 < = v ) \ / ( u < = 0 / \ v < = 0 ) - > rabs ( avg _ half _ sub u v - ( ( u + v ) / 2 ) ) < = 3 / 2 * ulp _ flt ( ( u + v ) / 2 ).
Match Accuracy:  14.89%
--------------------------------------------------------------------------------
Generated Proof: lemma ( _ zero : a = 0 - > av = 0.
Actual Proof:    lemma average _ zero : a = 0 - > av = 0.
Match Accuracy:  92.31%
--------------------------------------------------------------------------------


Evaluating: 50it [00:10,  4.68it/s]

Generated Proof: lemma round _ ne _ pt _ pos : forall x, ( 0 < x ) % r - >d _ ne _ pt x ( round beta =xp z ye x ).
Actual Proof:    lemma round _ ne _ pt _ pos : forall x, ( 0 < x ) % r - > rnd _ ne _ pt x ( round beta fexp zneareste x ).
Match Accuracy:  63.64%
--------------------------------------------------------------------------------

Exact Match Accuracy: 32.00%
Average BLEU Score:   0.7838
Average METEOR Score: 0.8954
Average ROUGE Score:  1.7147
Average Match Accuracy: 61.70%


In [ ]:
!pip install transformers rouge_score nltk tqdm

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=13ca2d165d7f98a2754e30fcdc556f09cdba1a15f8f5903e3b23cfd4a7663fca
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving combined_dataset.parquet to combined_dataset.parquet


In [ ]:
!apt-get install coq -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  ledit libcoq-core-ocaml libcoq-stdlib libfindlib-ocaml libfindlib-ocaml-dev
  libzarith-ocaml ocaml ocaml-base ocaml-compiler-libs ocaml-findlib
  ocaml-interp ocaml-man ocaml-nox
Suggested packages:
  coqide | proofgeneral libcoq-core-ocaml-dev why coq-doc ocaml-doc
  elpa-tuareg camlp4
The following NEW packages will be installed:
  coq ledit libcoq-core-ocaml libcoq-stdlib libfindlib-ocaml
  libfindlib-ocaml-dev libzarith-ocaml ocaml ocaml-base ocaml-compiler-libs
  ocaml-findlib ocaml-interp ocaml-man ocaml-nox
0 upgraded, 14 newly installed, 0 to remove and 30 not upgraded.
Need to get 281 MB of archives.
After this operation, 986 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcoq-stdlib amd64 8.15.0+dfsg-2 [24.7 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd6

In [ ]:
!pip install torch_geometric